In [1]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd

!pip install geopy
from geopy.geocoders import Nominatim

from IPython.display import Image
from IPython.core.display import HTML

from pandas.io.json import json_normalize

!pip install folium==0.5.0
import folium 

import numpy as np

from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib
import matplotlib.pyplot as plt

!pip install uszipcode
from uszipcode import SearchEngine

import os
from sklearn.cluster import DBSCAN


print('packages installed')

     |████████████████████████████████| 79 kB 9.3 MB/s  eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=b254c6c09b2505ffa4ffee04bcb4686674b469f0db37da5ba58c19cfc345cf13
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium
     |████████████████████████████████| 378 kB 15.1 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 14.5 MB/s eta 0:00:01
     |████████████████████████████████| 44 kB 6.2 MB/s  eta 0:00:01
     |████████████████████████████████| 41 kB 732 kB/s  eta 0:00:01
packages installed


In [2]:
response = requests.get("http://www.healthysf.org/bdi/outcomes/zipmap.htm")
soup = BeautifulSoup(response.text, "lxml")
table = soup.find_all("table")
df = pd.read_html(str(table))
df = pd.DataFrame(df[4])
df.columns = df.iloc[0]
df = df.iloc[1:-1, :-1]
sf_data = df
sf_data.head()

,Zip Code,Neighborhood
1,94102,Hayes Valley/Tenderloin/North of Market
2,94103,South of Market
3,94107,Potrero Hill
4,94108,Chinatown
5,94109,Polk/Russian Hill (Nob Hill)


In [3]:

search = SearchEngine(simple_zipcode=True)

latitude = []
longitude = []

for index, row in df.iterrows():
    zipcode = search.by_zipcode(row["Zip Code"]).to_dict()
    latitude.append(zipcode.get("lat"))
    longitude.append(zipcode.get("lng"))

sf_data["Latitude"] = latitude
sf_data["Longitude"] = longitude

sf_data.head()

Start downloading data for simple zipcode database, total size 9MB ...
  1 MB finished ...
  2 MB finished ...
  3 MB finished ...
  4 MB finished ...
  5 MB finished ...
  6 MB finished ...
  7 MB finished ...
  8 MB finished ...
  9 MB finished ...
  10 MB finished ...
  Complete!


,Zip Code,Neighborhood,Latitude,Longitude
1,94102,Hayes Valley/Tenderloin/North of Market,37.780,-122.420
2,94103,South of Market,37.780,-122.410
3,94107,Potrero Hill,37.770,-122.390
4,94108,Chinatown,37.791,-122.409
5,94109,Polk/Russian Hill (Nob Hill),37.790,-122.420


In [4]:
address = 'San Francisco'

geolocator = Nominatim(user_agent = "san_francisco_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of San Francisco are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of San Francisco are 37.7790262, -122.4199061.


In [108]:
# @hidden_cell
CLIENT_ID = 'XG4HLFPUBOFV1QEWJZR310AERKYQVUQT5CAFQ23HYULC3OQW'
CLIENT_SECRET = 'HOBXJIBPTDLNW1EDZG41B0GS2CSECAVAOWFQKDKN335WVLSX'
ACCESS_TOKEN = 'SJNUUKCZY4IOXQ5Z5HLZY31KNW450XCXPN4VQD1CYMQ23MZN'
VERSION = '20180604'
LIMIT = 95000
print(CLIENT_ID)
print(CLIENT_SECRET)

XG4HLFPUBOFV1QEWJZR310AERKYQVUQT5CAFQ23HYULC3OQW
HOBXJIBPTDLNW1EDZG41B0GS2CSECAVAOWFQKDKN335WVLSX


In [112]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
      
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
       
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

sf_venues = getNearbyVenues(names = sf_data['Neighborhood'],
                                   latitudes = sf_data['Latitude'],
                                   longitudes = sf_data['Longitude']
                                  )
sf_venues.filter(["Venue", "Venue Latitude", "Venue Longitude","Venue Category"]) 
                                  

,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Herbst Theater,37.779548,-122.420953,Concert Hall
1,War Memorial Opera House,37.778601,-122.420816,Opera House
2,Louise M. Davies Symphony Hall,37.777976,-122.420157,Concert Hall
3,San Francisco Ballet,37.778580,-122.420798,Dance Studio
4,War Memorial Court,37.779042,-122.420971,Park
...,...,...,...,...
1759,Zen,37.721804,-122.419453,Trail
1760,Mcclaren Mountain Mystery Maze,37.719990,-122.420320,Outdoor Sculpture
1761,Taqueria El Buen Sebor,37.711979,-122.405557,Mexican Restaurant
1762,TGIF Beauty Supply,37.727345,-122.403738,Cosmetics Shop


In [111]:

sf_venues = sf_venues[['Venue', 'Venue Latitude', 'Venue Longitude']]
sf_venues = sf_venues.set_index('Venue')
sf_venues.head()

KeyError: "['Venue'] not in index"

In [8]:
sf_venues.values

array([[  37.77954818, -122.42095294],
       [  37.77860106, -122.42081555],
       [  37.77797616, -122.42015695],
       ...,
       [  37.711945  , -122.405906  ],
       [  37.71202087, -122.40522766],
       [  37.7242812 , -122.4007499 ]])

sf_venues.plot.scatter(x = 'latitude', y = 'longitude', c=labels, s=50, cmap='viridis')
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5)
plt.title('Clustering GPS Co-ordinates to Form Regions - Weighted',fontsize=18, fontweight='bold')

In [9]:
kmeans = KMeans(n_clusters = 6, max_iter=100, init ='k-means++')
lat_long = sf_venues[sf_venues.columns[1:3]]
lot_size = sf_venues[sf_venues.columns[1]]
weighted_kmeans_clusters = kmeans.fit(lat_long, sample_weight = lot_size) # Compute k-means clustering.
sf_venues['cluster_label'] = kmeans.predict(lat_long, sample_weight = lot_size)
centers = kmeans.cluster_centers_ # Coordinates of cluster centers.
labels = sf_venues['cluster_label'] # Labels of each point

In [10]:
sf_venues.loc[sf_venues['cluster_label'] == 1, sf_venues.columns[[0,1]+ list(range(5, sf_venues.shape[1]))]]

,Venue Latitude,Venue Longitude
Venue,,
Marco Polo Italian Ice Cream,37.742758,-122.481856
Lou's Cafe,37.742805,-122.482368
Dumpling Specialist,37.742832,-122.478320
House of Pancakes,37.742856,-122.476184
ODSY Workshop,37.742550,-122.481422
...,...,...
19th Ave & Junipero Serra Blvd,37.717039,-122.472461
Lake Merced Fishing Pier And Nature Trail,37.713396,-122.486065
Stonestown MUNI Metro Platform,37.727280,-122.474935


In [11]:
sf_venues.loc[sf_venues['cluster_label'] == 2, sf_venues.columns[[0,1]+ list(range(5, sf_venues.shape[1]))]]

,Venue Latitude,Venue Longitude
Venue,,
AJ's Barbeque Cafe,37.720757,-122.446280
City College: Community Health & Wellness Center,37.723996,-122.449445
Crazy Pepper,37.720832,-122.446663
MUNI Bus Stop - Geneva & San Jose,37.720451,-122.446867
Balboa Park Bart: Food Truck,37.721191,-122.447329
...,...,...
Sessions at the Presidio,37.799154,-122.447889
Letterman Digital Arts Center,37.799408,-122.449515
Little Marina Green,37.805478,-122.447863


In [12]:
sf_venues.loc[sf_venues['cluster_label'] == 3, sf_venues.columns[[0,1]+ list(range(10, sf_venues.shape[1]))]]

,Venue Latitude,Venue Longitude
Venue,,
Stagecoach Greens,37.770867,-122.390261
SPARK Social SF,37.770762,-122.391689
Spro Coffeelab,37.770835,-122.391400
SFFSoccer Mission Bay Field,37.770886,-122.392197
Mission Bay Kids Park,37.772068,-122.392548
...,...,...
KC Custom Builders,37.725413,-122.381908
India Basin,37.734312,-122.376112
india basin open space,37.733372,-122.371933


In [13]:
sf_venues.loc[sf_venues['cluster_label'] == 4, sf_venues.columns[[0,1]+ list(range(5, sf_venues.shape[1]))]]

,Venue Latitude,Venue Longitude
Venue,,
The Strand,37.779888,-122.413138
Arsicault Bakery,37.780789,-122.413433
City Beer Store,37.778493,-122.412244
Alonzo King LINES Dance Center,37.780116,-122.412187
Birdsong,37.779425,-122.410473
...,...,...
Pho Luen Fat,37.712113,-122.405905
Chef Li's Cafe,37.727191,-122.403772
Buffalo Kitchen,37.711945,-122.405906


In [14]:
sf_venues.loc[sf_venues['cluster_label'] == 5, sf_venues.columns[[0,1]+ list(range(5, sf_venues.shape[1]))]]

,Venue Latitude,Venue Longitude
Venue,,
Olive This Olive That,37.751477,-122.428590
Saru Sushi Bar,37.751647,-122.428774
Noe Valley Farmer's Market,37.751471,-122.428988
Taquerias El Farolito,37.721230,-122.437395
Tao Sushi,37.721037,-122.437665
...,...,...
Fitness Court,37.806072,-122.435177
The Italian Homemade Company,37.797628,-122.430994
Yoga Flow SF - Union,37.797848,-122.430471


In [92]:
kclusters = 5
#sf_grouped_clustering = sf_venues.drop('Venue', 1)
# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(sf_venues)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [105]:
# create map
map_clusters = folium.Map(location = [latitude, longitude], zoom_start = 11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i * x) ** 2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, cluster in zip(sf_venues['Venue Latitude'], sf_venues['Venue Longitude'], sf_venues['cluster_labels']):
    label = folium.Popup(str() + ' Cluster ' + str(cluster), parse_html=True) 
    folium.CircleMarker(
        [lat, lon],
        radius = 3,
        popup = label,
        color = rainbow[cluster - 1],
        fill = True,
        fill_color = rainbow[cluster - 1],
        fill_opacity = 0.5).add_to(map_clusters)
       
map_clusters